# 1. CI/CD para Modelos de ML

### O treinamento de um modelo de machine learning

Antes de qualquer escolha de modelo de ML, é primordial entender o problema de negócio junto aos usuários. A implementação de um algoritmo consiste em horas de desenvolvimento, testes, retreinamento e monitoramento, o que pode ser oneroso se o caso não for bem estudado de início.

Considerando que o business case foi avaliado propriamente e a implementação de um modelo de clustering corresponde às demandas de negócio. Usando o framework `sklearn`, o passo-a-passo, no meu ponto de vista, seria:

1. **Ter um dataset de treino preparado**. Se for uma lista de dicionários, por exemplo, será necessário convertê-lo em um `numpy.ndarray` para usarmos na entrada do algoritmo de clustering.

2. **Definir os hiperparâmetros** específicos do algoritmo de ML escolhido.  
   Como escolhemos clustering, será necessário estudar a quantidade ideal de grupos. Para isso podemos utilizar técnicas como o *Elbow Method*, por exemplo. Dessa forma, teremos nosso número de clusters.

3. **Adicionar regras de negócio ao modelo**, como delimitação do raio de cada centróide do cluster e afins.

4. **Visualizar o resultado do modelo**. Após o treinamento, o cientista pode plotar o resultado do modelo junto aos seus clusters em um mapa utilizando frameworks como `Folium`. Seria interessante, pois a visualização do modelo e clusters via `Folium` pode ser personalizada com cores, tornando-se entendível até para os usuários.

5. **Salvar o modelo e registrar os artefatos do experimento (ainda em DEV)**.  
   Esta etapa é importante, pois o modelo será usado futuramente no pipeline de inferência. Plataformas open-source como `MLflow` já implementam práticas e tarefas de registro e deployment de modelos de machine learning. No entanto, a depender da empresa, outras plataformas podem realizar o registro dos artefatos. O `Databricks` se destaca. A AWS também possui esse serviço. Podemos salvar os artefatos manualmente da mesma forma, principalmente quando não é necessário criar modelos de machine learning em massa.

Dessa forma, criamos o modelo, treinamos, salvamos e registramos o experimento e seus artefatos utilizando o `MLflow`, etapa condizente para eventuais auditorias e fallback.

Este pipeline de treino pode ser automatizado via script. O script, salvo no diretório principal do projeto, executa os dados de treino, artefatos e todo o processo descrito acima, salvando o nome do experimento no `MLflow`, por exemplo.

## A implantação em produção

Usando ferramentas como `Docker`, `Kubernetes` ou plataformas em nuvem, é possível colocar o modelo em produção. É preciso ficar claro que não existe uma única forma de prosseguir com o processo de produção da modelagem após a análise exploratória do projeto. Existem dois cenários muito comuns que podemos escolher:

1. **Deployment manual do modelo**, criando uma API (do zero) e "deployando" em um ambiente escalável (`Kubernetes`).
2. **Deployment do modelo por meio de plataformas que facilitam MLOps**, utilizando modelos de *model serving*.

Abaixo, temos um design de referência para o pipeline de inferência (*online serving*).

<p align="center">
  <img src="00_images/Imagem1.png"/>
</p>

# Simulando um processo de CI/CD do GitHub Actions no localhost

Vamos simular um processo de CI/CD do GitHub Actions no localhost de forma open-source. Iremos chamar o modelo através de uma API. 

## Passo 1: Gerar a versão da API

Esta API deve ser desenvolvida com todos os parâmetros `path` correspondendo à arquitetura levantada na questão 1. Após o desenvolvimento, deve-se:

1. Criar um novo repositório no GitHub (por ser um projeto open-source, mas o Bitbucket também poderia ser usado).
2. Adicionar o código da API na branch `main`.

## Deploy da API

Para o deploy da API, as opções incluem:

1. **Docker com Kubernetes**: Ideal para maior escalabilidade.
2. **Render**: Uma plataforma mais barata e amigável para hospedar aplicações.

### Deploy da API no Render

A forma mais simples para fazer o deploy da API no Render é utilizando o seguinte template:  
[Template Render FastAPI](https://github.com/new?template_name=fastapi&template_owner=render-examples).  

1. Escolha o repositório GitHub da sua API.  
2. Defina o seguinte comando para iniciar sua aplicação:  

    ```bash
   puvicorn main_render:app --host 0.0.0.0 --port $PORT 

Importante salientar que, se procuramos escalabilidade e disponibilidade, o Render pode não ser a opção ideal, sendo o kubernetes mais apropriado para a tarefa, a depender do budget do projeto. Neste caso, podemos copiar o modelo para o bucket no S3 para maior versatilidade de ferramentas hospedagem, visto que o MLflow tem problemas de escalabilidade, apesar de fornecer recursos de model serving.

# 2. CI/CD para Modelos de ML

O código para aplicação está todo na pasta 03_scripts. Abaixo pode conferir a API funcionando

In [7]:
import numpy as np
import requests

In [8]:
response = requests.get("http://127.0.0.1:5000/health")
print(response.text)  # Saída esperada: "Serviço em execução"

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000208D4CAC5F0>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))

In [ ]:
# Exemplo de entradas x e y
x = np.array([0.82737724, -0.73769513, -0.44605037, 0.43279337, -0.47367361,
              -0.50244517, 0.58595414, 0.05915988, -0.09502409])

y = np.array([-1.56610693, 1.35557354, 0.71503732, 0.43279337, -0.47367361,
              0.78684529, -1.9423032, 0.05915988, 1.16865443])

# Convertendo as entradas para listas
data = {
    "inputs": [x.tolist(), y.tolist()]  # Enviando múltiplas entradas
}

url = "http://127.0.0.1:5000/predict"
headers = {"Content-Type": "application/json"}

response = requests.post(url, json=data, headers=headers)
if response.status_code == 200:
    print("Previsões:", response.json())
else:
    print(f"Erro: {response.status_code}, {response.text}")